In [1]:
import os
import argparse
import pandas as pd
from tqdm import tqdm
from termcolor import colored
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
import sys
sys.path.append('..')

/home/sam/anaconda3/envs/QA_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_csv(file_name:str):
    file_path = os.path.join(os.getcwd(), file_name)
    return pd.read_csv(file_path)

def measure_accuracy(LLM, prompt, Q, A, RAG:bool=False):
    correct_count = 0
    wrong_count = 0
    unsure_count = 0
    for q,a in tqdm(zip(Q, A), total=len(Q), desc="Measuring Accuracy"):
        if RAG:
            pred = LLM.invoke(prompt + q)['result']
        else:
            pred = LLM.invoke(prompt + q).content
        pred = pred.strip()[0]
        print(f"Correct Answer: {a}, Predicted Answer: {pred}")

        if pred == 'X': # if the answer is X (don't know)
            unsure_count += 1
        else:
            if a == pred:
                correct_count += 1
            else:
                wrong_count += 1
    print(colored(f"Correct: {correct_count}/{len(Q)}", 'green'))
    print(colored(f"Wrong: {wrong_count}/{len(Q)}", 'red'))
    print(colored(f"Unsure: {unsure_count}/{len(Q)}", 'yellow'))
    return correct_count, wrong_count, unsure_count
        

def get_llm_config(params:dict):
    LLM_name = "mistral:instruct" # https://ollama.com/library/mistral:instruct
    LLM = ChatOllama(model=LLM_name, temperature=params['temperature'])
    params['llm'] = LLM_name
    params['is_hf'] = False
    return LLM, params

def get_llm(params:dict):
    LLM, config = get_llm_config(params)
    if params['is_hf']:
        pipe = pipeline(
            task = params['task'],
            model = LLM,
            tokenizer = params['tokenizer'],
            pad_token_id = params['tokenizer'].eos_token_id,
            max_length = params['max_length'],
            temperature = params['temperature'],
            do_sample = params['do_sample'] if params['task'] == 'text2text-generation' else None,
            top_p = params['top_p'] if params['task'] == 'text-generation' else None,
            repetition_penalty = params['repetition_penalty']
            )
        LLM = HuggingFacePipeline(pipeline = pipe)
    return LLM, config

### Multiple Choce Questions (MCQs)

In [3]:
# MCQs about Omicron
MCQs_omicrons = read_csv("MCQs_omicron.csv")
MCQs_omicrons_Q = MCQs_omicrons['Q']
MCQs_omicrons_A = MCQs_omicrons['A']

### Model without RAG

In [4]:
# model parameters
params = {
    'chain_type': 'stuff',
    'embedding_device': 'cuda',
    'embedding_model': 'sentence-transformers/all-MiniLM-L6-v2',
    'k': 3,
    'llm': 'google/flan-t5-base',
    'llm_device': 'cuda',
    'max_length': 2000,
    'quantize': True,
    'query_instruction': 'Represent the question for retrieving supporting documents',
    'repetition_penalty': 1.0,
    'search_type': 'similarity',
    'separator': '\n\n',
    'temperature': 0.05,
    'top_p': 1.0,
    'chunk_size': 500,
    'chunk_overlap': 0,
}

# engineered prompt template
prompt = """You are the angent that has to select the correct answer to the following multiple choice question in the context provided.
            You cannot speak human language, but you can only say one single letter.
            Choose the letter corresponding to the correct answer.
            If you don't know or unsure about the answer, just display the letter X without any additional text.
            If you know the answer, display the letter corresponding to the correct answer without any additional text.
            Whether you know or don't know the answer, do not display any other texts except for the letter.
            Your response always has to be just one single letter.
            Example of the answer that you have to say: A.
            Context is: """

In [5]:
# Large Language Model (LLM)
LLM, _ = get_llm(params)
print(colored(f"MCQ Generator LLM: {LLM}", "yellow"))

MCQ Generator LLM: model='mistral:instruct' temperature=0.05


In [6]:
# measure accuracy
acc = measure_accuracy(LLM, prompt, MCQs_omicrons_Q, MCQs_omicrons_A)

Measuring Accuracy:   4%|▍         | 1/25 [00:09<03:50,  9.61s/it]

Correct Answer: C, Predicted Answer: C


Measuring Accuracy:   8%|▊         | 2/25 [00:21<04:11, 10.95s/it]

Correct Answer: B, Predicted Answer: C


Measuring Accuracy:  12%|█▏        | 3/25 [00:35<04:30, 12.29s/it]

...

Correct Answer: C, Predicted Answer: C


Measuring Accuracy:  96%|█████████▌| 24/25 [04:02<00:10, 10.49s/it]

Correct Answer: A, Predicted Answer: X


Measuring Accuracy: 100%|██████████| 25/25 [04:09<00:00,  9.97s/it]

Correct Answer: C, Predicted Answer: C
Correct: 11/25
Wrong: 2/25
Unsure: 12/25


### Model with RAG

In [7]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings, HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter, TokenTextSplitter, NLTKTextSplitter, SpacyTextSplitter
from langchain_community.vectorstores import Cassandra, Chroma, FAISS # vector database
from utils import pdf_loader, docs_splitter, get_embeddings, \
                    build_database, get_retriever, get_qa_chain

In [8]:
def RAG(pdf:str, params:dict):
    # get LLM
    LLM, config = get_llm(params)
    # load document
    pdf = os.path.join("../data", pdf) # PDF file
    assert pdf.endswith(".pdf"), "Please provide a PDF document"
    loader = PyPDFLoader(pdf) # PDF loader
    docs = loader.load() # load document
    # split document
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=params['chunk_size'], chunk_overlap=params['chunk_overlap'], separators=params['separator'])
    chunks = docs_splitter(text_splitter=text_splitter, docs=docs)
    # get embeddings
    embedding_model_wrapper = HuggingFaceInstructEmbeddings
    embedding_model = params['embedding_model']
    embeddings = get_embeddings(embedding_model_wrapper, embedding_model, device=params['embedding_device'], query_instruction=params['query_instruction'])
    # build database
    database = FAISS
    db = build_database(database, chunks, embeddings)
    # get retriever
    retriever = get_retriever(db, search_type=params['search_type'], k=params['k'])
    # get Q&A chain
    qa_chain = get_qa_chain(LLM, retriever, chain_type=params['chain_type'])
    return qa_chain

In [9]:
pdf = "Omicron Variant Symptoms and Treatment.pdf"
qa_chain = RAG(pdf, params)

Document has been split into 31 chunks
load INSTRUCTOR_Transformer


/home/sam/anaconda3/envs/QA_llm/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


max_seq_length  512
Embeddings have been generated using HuggingFaceInstructEmbeddings
Building FAISS vector database...
FAISS vector database has successfully been built
Vector retriever has been created for similarity search
Q&A chain has been created


In [10]:
# measure accuracy
acc = measure_accuracy(qa_chain, prompt, MCQs_omicrons_Q, MCQs_omicrons_A, RAG=True)

Measuring Accuracy:   4%|▍         | 1/25 [00:10<04:22, 10.94s/it]

Correct Answer: C, Predicted Answer: C


Measuring Accuracy:   8%|▊         | 2/25 [00:13<02:20,  6.11s/it]

Correct Answer: B, Predicted Answer: B


Measuring Accuracy:  12%|█▏        | 3/25 [00:17<01:54,  5.19s/it]

...

Correct Answer: C, Predicted Answer: C


Measuring Accuracy:  96%|█████████▌| 24/25 [05:31<00:15, 15.31s/it]

Correct Answer: A, Predicted Answer: A


Measuring Accuracy: 100%|██████████| 25/25 [05:46<00:00, 13.86s/it]

Correct Answer: C, Predicted Answer: C
Correct: 19/25
Wrong: 3/25
Unsure: 3/25
